In [9]:
import gc
from copy import deepcopy
import cv2
import numpy as np
from random import randint

from keras.preprocessing.image import ImageDataGenerator

import matplotlib
import matplotlib.pyplot as plt

from util import random_mask
from keras.models import Model
from keras.layers import Input

# Settings
MAX_BATCH_SIZE = 128

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Construct Model

In [20]:
def get_model():
    img_rows = img_cols = 256
    input1 = Input((img_rows, img_cols, 3), name='input1')
    input2 = Input((img_rows, img_cols, 1), name='input2')
    
    output = input1
    model = Model(inputs=[input1, input2], outputs=output)
    return model

# Script to Generate Random Mask

In [21]:
def random_mask(height, width, channels=3):
    """Generates a random irregular mask with lines, circles and elipses"""    
    img = np.zeros((height, width, channels), np.uint8)

    # Set size scale
    size = int((width + height) * 0.03)
    if width < 64 or height < 64:
        raise Exception("Width and Height of mask must be at least 64!")
    
    # Draw random lines
    for _ in range(randint(1, 20)):
        x1, x2 = randint(1, width), randint(1, width)
        y1, y2 = randint(1, height), randint(1, height)
        thickness = randint(3, size)
        cv2.line(img,(x1,y1),(x2,y2),(1,1,1),thickness)
        
    # Draw random circles
    for _ in range(randint(1, 20)):
        x1, y1 = randint(1, width), randint(1, height)
        radius = randint(3, size)
        cv2.circle(img,(x1,y1),radius,(1,1,1), -1)
        
    # Draw random ellipses
    for _ in range(randint(1, 20)):
        x1, y1 = randint(1, width), randint(1, height)
        s1, s2 = randint(1, width), randint(1, height)
        a1, a2, a3 = randint(3, 180), randint(3, 180), randint(3, 180)
        thickness = randint(3, size)
        cv2.ellipse(img, (x1,y1), (s1,s2), a1, a2, a3,(1,1,1), thickness)
    
    return 1-img

In [22]:
# Load image
img = cv2.imread('tensorBoardResults.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (256, 256)) / 255
img_shape = img.shape
print(f"Shape of image is: {img_shape}")

# Load mask
mask = random_mask(shape[0], shape[1], 1)
mask_shape = mask.shape
print(f"Shape of mask is: {mask_shape}")

Shape of image is: (256, 256, 3)
Shape of mask is: (256, 256, 1)


## Creating data generator

In [23]:
class DataGenerator(ImageDataGenerator):
    def flow(self, x, *args, **kwargs):
        while True:
            
            # Get augmentend image samples
            ori = next(super().flow(x, *args, **kwargs))
            print('ori shape: {}'.format(ori.shape))
            # Get masks for each image sample
            #mask = np.stack([random_mask(ori.shape[1], ori.shape[2], 1) for _ in range(ori.shape[0])], axis=0)
            mask = np.stack([random_mask(ori.shape[1], ori.shape[2]) for _ in range(ori.shape[0])], axis=0)

            print('mask shape: {}'.format(mask.shape))

            # Yield ([ori, masl],  ori) training batches
            # print(masked.shape, ori.shape)
            gc.collect()
            yield [ori, mask], ori     

# Create datagen
datagen = DataGenerator(  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Create generator from numpy arrays
batch = np.stack([img for _ in range(MAX_BATCH_SIZE)], axis=0)

generator = datagen.flow(x=batch, batch_size=4)

# Get samples & Display them
[ori, mask], ori = next(generator)

# Show side by side
# _, axes = plt.subplots(1, 3, figsize=(20, 5))
# axes[0].imshow(masked[0,:,:,:])
# axes[1].imshow(mask[0,:,:,:]*255)
# axes[2].imshow(ori[0,:,:,:])

ori shape: (4, 256, 256, 3)
mask shape: (4, 256, 256, 3)


## Training classifier on single image

In [24]:
def plot_callback(model):
    """Called at the end of each epoch, displaying our previous test images,
    as well as their masked predictions and saving them to disk"""
    print('plot_callback called...')
    # Get samples & Display them        
    pred_img = model.predict([ori, mask])
    print(pred_img.shape)

    # Clear current output and display test images
#     for i in range(len(ori)):
#         _, axes = plt.subplots(1, 3, figsize=(20, 5))
#         axes[0].imshow(masked[i,:,:,:])
#         axes[1].imshow(pred_img[i,:,:,:] * 1.)
#         axes[2].imshow(ori[i,:,:,:])
#         axes[0].set_title('Masked Image')
#         axes[1].set_title('Predicted Image')
#         axes[2].set_title('Original Image')                
#         plt.show()

In [25]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('model-dsbowl2018-1.h5', verbose=1, save_best_only=True)
test_model = get_model()
test_model.compile(optimizer='sgd', loss='mse')
test_model.fit(
    generator, 
    steps_per_epoch=1,
    epochs=1,
    callbacks=plot_callback
    #callbacks=[earlystopper, checkpointer]
)

AttributeError: 'generator' object has no attribute 'ndim'